In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint, Eq, solve, sqrt
from sympy import sin, cos, asin, acos, pi, lambdify
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify

from scipy.integrate import solve_ivp

import numpy as np

from IPython import display # for the animation

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
plt.ion()

import warnings
warnings.filterwarnings('ignore')

In [2]:
def T(theta, x, y):
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

# Simulating a multi-link system

In [3]:
(t, theta_1, theta_2, theta_3, l_1, l_2, l_3, m_1, m_2, m_3, g) = symbols('t, theta_1 theta_2 theta_3 l_1 l_2 l_3 m_1 m_2 m_3 g', real = True)

theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [4]:
T_01 = T(theta_1, l_1, 0)
T_12 = T(theta_2, l_2, 0)
T_23 = T(theta_3, l_3, 0)

In [5]:
FK1 = (T_01 * Matrix([[l_1], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[l_2], [0], [1]]))[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[l_3], [0], [1]]))[:-1,0])

In [6]:
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
J = Matrix([[J1], [J2], [J3]])

In [7]:
mass_xy = Matrix([[m_1, 0, 0, 0, 0, 0],
                  [0, m_1, 0, 0, 0, 0],
                  [0, 0, m_2, 0, 0, 0],
                  [0, 0, 0, m_2, 0, 0],
                  [0, 0, 0, 0, m_3, 0],
                  [0, 0, 0, 0, 0, m_3]])

M_jacobian = simplify(J.T * mass_xy * J)
M_jacobian

Matrix([
[l_1**2*m_1 + 2*l_2**2*m_2*cos(theta_2(t)) + 2*l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_2(t) + theta_3(t)) + 2*l_2*l_3*m_3*cos(theta_2(t)) + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3, l_2**2*m_2*cos(theta_2(t)) + l_2**2*m_2 + l_2*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2*l_3*m_3*cos(theta_2(t)) + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3, l_3*m_3*(l_2*cos(theta_2(t) + theta_3(t)) + l_3*cos(theta_3(t)) + l_3)],
[                                  l_2**2*m_2*cos(theta_2(t)) + l_2**2*m_2 + l_2*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2*l_3*m_3*cos(theta_2(t)) + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3,                                                                                                       l_2**2*m_2 + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3,                                       l_3**2*m_3*(cos(theta_3(t)) + 1)],
[                                                                                                                          l_3*m_3*(l_2

In [8]:
v1 = diff(FK1,t)
v2 = diff(FK2,t)
v3 = diff(FK3,t)

KE1 = (1/2)*m_1*(v1.T * v1)[0]
KE2 = (1/2)*m_2*(v2.T * v2)[0]
KE3 = (1/2)*m_3*(v3.T * v3)[0]

In [9]:
T = KE1 + KE2 + KE3
V = m_1*g*FK1[1] + m_2*g*FK2[1] + m_3*g*FK3[1]
L = T - V
L = simplify(L)

In [10]:
EOM_theta_1 = diff(diff(L, Derivative(theta_1, t)), t) - diff(L, theta_1)
EOM_theta_1 = nsimplify(EOM_theta_1)

EOM_theta_2 = diff(diff(L, Derivative(theta_2, t)), t) - diff(L, theta_2)
EOM_theta_2 = nsimplify(EOM_theta_2)

EOM_theta_3 = diff(diff(L, Derivative(theta_3, t)), t) - diff(L, theta_3)
EOM_theta_3 = nsimplify(EOM_theta_3)

In [11]:
mass = sp.symarray('',(3,3))

mass[0,0] = EOM_theta_1.expand().coeff(Derivative(theta_1,t,t))
mass[0,1] = EOM_theta_1.expand().coeff(Derivative(theta_2,t,t))
mass[0,2] = EOM_theta_1.expand().coeff(Derivative(theta_3,t,t))

mass[1,0] = EOM_theta_2.expand().coeff(Derivative(theta_1,t,t))
mass[1,1] = EOM_theta_2.expand().coeff(Derivative(theta_2,t,t))
mass[1,2] = EOM_theta_2.expand().coeff(Derivative(theta_3,t,t))

mass[2,0] = EOM_theta_3.expand().coeff(Derivative(theta_1,t,t))
mass[2,1] = EOM_theta_3.expand().coeff(Derivative(theta_2,t,t))
mass[2,2] = EOM_theta_3.expand().coeff(Derivative(theta_3,t,t))

mass = Matrix(mass)
mass = simplify(mass)
mass

Matrix([
[l_1**2*m_1 + 2*l_2**2*m_2*cos(theta_2(t)) + 2*l_2**2*m_2 + l_2**2*m_3 + 2*l_2*l_3*m_3*cos(theta_2(t) + theta_3(t)) + 2*l_2*l_3*m_3*cos(theta_2(t)) + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3, l_2**2*m_2*cos(theta_2(t)) + l_2**2*m_2 + l_2*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2*l_3*m_3*cos(theta_2(t)) + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3, l_3*m_3*(l_2*cos(theta_2(t) + theta_3(t)) + l_3*cos(theta_3(t)) + l_3)],
[                                  l_2**2*m_2*cos(theta_2(t)) + l_2**2*m_2 + l_2*l_3*m_3*cos(theta_2(t) + theta_3(t)) + l_2*l_3*m_3*cos(theta_2(t)) + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3,                                                                                                       l_2**2*m_2 + 2*l_3**2*m_3*cos(theta_3(t)) + 2*l_3**2*m_3,                                       l_3**2*m_3*(cos(theta_3(t)) + 1)],
[                                                                                                                          l_3*m_3*(l_2

In [12]:
M_jacobian == mass

True

In [13]:
accel_state_vector = Matrix([[Derivative(theta_1, t, t)], [Derivative(theta_2, t, t)], [Derivative(theta_3, t, t)]])

EOM1_leftover = simplify(EOM_theta_1.expand() - (mass[0,:]*accel_state_vector)[0])
EOM2_leftover = simplify(EOM_theta_2.expand() - (mass[1,:]*accel_state_vector)[0])
EOM3_leftover = simplify(EOM_theta_3.expand() - (mass[2,:]*accel_state_vector)[0])
# EOM1_leftover

In [14]:
G_1 = EOM1_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_2 = EOM2_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])
G_3 = EOM3_leftover.subs([(Derivative(theta_1, t), 0), (Derivative(theta_2, t), 0), (Derivative(theta_3, t), 0)])

In [15]:
C_1 = EOM1_leftover - G_1
C_2 = simplify(EOM2_leftover - G_2)
C_3 = simplify(EOM3_leftover - G_3)

In [16]:
simplify(Matrix([[EOM_theta_1], [EOM_theta_2], [EOM_theta_3]]) - (mass*accel_state_vector + Matrix([[C_1], [C_2], [C_3]]) + Matrix([[G_1], [G_2], [G_3]])))

Matrix([
[0],
[0],
[0]])

In [17]:
M = lambdify((theta_1, theta_2, theta_3, 
              Derivative(theta_1, t), Derivative(theta_2, t), Derivative(theta_3, t), 
              l_1, l_2, l_3, m_1, m_2, m_3), mass)
EOM = lambdify((theta_1, theta_2, theta_3, 
                Derivative(theta_1, t), Derivative(theta_2, t), Derivative(theta_3, t), 
                l_1, l_2, l_3, m_1, m_2, m_3), 
               Matrix([[EOM1_leftover.subs([(g, 9.8)])], [EOM2_leftover.subs([(g, 9.8)])], [EOM3_leftover.subs([(g, 9.8)])]]))

In [18]:
def dynamics(t, state, l_1, l_2, l_3,  m_1, m_2, m_3):
    theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3 = state
    
    dydt = np.linalg.inv(M(theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3, l_1, l_2, l_3, m_1, m_2, m_3))@(-EOM(theta_1, theta_2, theta_3, dtheta_1, dtheta_2, dtheta_3, l_1, l_2, l_3, m_1, m_2, m_3))

    return [dtheta_1, dtheta_2, dtheta_3, dydt[0][0], dydt[1][0], dydt[2][0]]

In [19]:
l1 = 1
l2 = 1
l3 = 1.5
m1 = 1
m2 = 1
m3 = 2

t_end = 10
dt = 0.001
time = np.linspace(0,t_end,int(t_end/dt))

sol = solve_ivp(lambda t, y: dynamics(t, y, l1, l2, l3, m1, m2, m3),
                [0,t_end], [np.pi/2, 0.0, 0.0, 0.0, 0.0, 0.0], 
                t_eval = time, 
                rtol=1e-8, atol = 1e-8)

plt.clf()
plt.plot(sol.t, sol.y[0,:], label='theta_1')
plt.plot(sol.t, sol.y[1,:], label='theta_2')
plt.plot(sol.t, sol.y[2,:], label='theta_3')
plt.legend()
plt.ylabel('joint angle')
plt.xlabel('time')

Text(0.5, 0, 'time')

In [ ]:
skip = 10

theta1 = sol.y[0,0:-1:skip]
theta2 = sol.y[1,0:-1:skip]
theta3 = sol.y[2,0:-1:skip]

for kk, (q1, q2, q3) in enumerate(zip(theta1, theta2, theta3)):
    plt.cla()
    # print(q1)
    
    plt.plot([0, l1*np.cos(q1)], [0, l1*np.sin(q1)], 'ko-')
    
    plt.plot([l1*np.cos(q1), l1*np.cos(q1) + l2*np.cos(q1 + q2)], [l1*np.sin(q1), l1*np.sin(q1)  + l2*np.sin(q1 + q2)], 'ko-')

    plt.plot([l1*np.cos(q1) + l2*np.cos(q1 + q2), l1*np.cos(q1) + l2*np.cos(q1 + q2) + l3*np.cos(q1 + q2 + q3)], 
              [l1*np.sin(q1)  + l2*np.sin(q1 + q2), l1*np.sin(q1) + l2*np.sin(q1 + q2) + l3*np.sin(q1 + q2 + q3)], 'ko-')
    
    plt.axis([-4,4,-4,4])

    plt.pause(0.00001)


# Rimless wheel with spring-loaded ankle

In [25]:
t, m, l, g, k, alpha, theta, omega, theta_0, omega_max = symbols('t m l g k alpha theta omega theta_0 omega_max')
theta = Function('theta', real=True)(t)
omega = Function('omega', real=True)(t)
k_constraint = k >= 0
theta_0_constraint = theta_0 <= -alpha

In [26]:
# The kinetic energy (the angular velocity ω)
I = m*l**2
T = 0.5*I*omega**2
# The potential energy
V = m*g*l*cos(theta)+0.5*k*(theta-theta_0)**2
# The Lagrangian
L = T-V
EL = diff(diff(L, omega), t) - diff(L, theta)
EL

-g*l*m*sin(theta(t)) + 0.5*k*(-2*theta_0 + 2*theta(t)) + 1.0*l**2*m*Derivative(omega(t), t)

In [27]:
initial_conditions = {theta: -alpha, omega: 0}
final_conditions = {theta: 0, omega: 0}
# The total mechanical energy
E = T + V
E0 = E.subs(initial_conditions)
Ef = E.subs(final_conditions)
conservation_equation = Eq(E0, Ef)
relationship = simplify(solve(conservation_equation, k)[0])
relationship

-2.0*g*l*m*(cos(alpha) - 1.0)/(alpha*(alpha + 2.0*theta_0))

In [28]:
initial_conditions = {theta: -alpha, omega: omega_max}
final_conditions = {theta: 0, omega: 0}
# The total mechanical energy
E = T + V
E0 = E.subs(initial_conditions)
Ef = E.subs(final_conditions)
conservation_equation = Eq(E0, Ef)
# Solve for the maximum initial angular velocity
omega_max = solve(conservation_equation, omega_max)

# Solve the constraint for circular motion (l*omega_max)^2 / l*g <= 1
constraint_equation = simplify(Eq((l*omega_max[0])**2 / l*g, 1))
constraint_equation

Eq(g*(1.0*alpha**2*k + 2.0*alpha*k*theta_0 + 2.0*g*l*m*(cos(alpha) - 1))/(l*m), -1)

In [29]:
omega_nn, omega_np, omega_n1 = symbols('omega_nn omega_np omega_n1')
# 0.5*m*l**2*omega_np**2 + m*g*l*cos(theta) + 0.5*k*(theta-theta_0)**2 = 0.5*m*l**2*omega_nn**2
# energy_equation = 0.5*m*l**2*omega_np**2 + m*g*l*cos(theta) + 0.5*k*(theta-theta_0)**2 - 0.5*m*l**2*omega_nn**2
# omega_nn_solution = solve(energy_equation, omega_nn)[0]
omega_nn = sqrt(omega_np**2 + 2*g*l*cos(theta) + (k*(theta-theta_0)**2)/(m*l**2))
reset_condition = omega_n1 - omega_nn * cos(2 * alpha)
touch_down_return_map = solve(reset_condition, omega_nn)[0]
touch_down_return_map

omega_n1/cos(2*alpha)

In [30]:
omega_star = symbols('omega_star')
touch_down_return_map = omega_star * cos(2 * alpha)
steady_state_speed = solve(omega_star - touch_down_return_map, omega_star)
steady_state_speed

[0]

In [20]:
m = 1
l = 5
g = 9.8
alpha = np.radians(30) 

k_values = np.linspace(0, 100, 300)
theta_0_values = np.linspace(-0.5*np.pi, -np.pi/6, 300)
K, Theta_0 = np.meshgrid(k_values, theta_0_values)

omega_star = np.sqrt((4*K*alpha*Theta_0) / (m*l**2) * ((np.cos(2*alpha)**2) / (np.cos(2*alpha)**2 - 1)))

# The robot cannot start from rest
condition1 = -0.5 * K * (alpha**2 + 2*alpha*Theta_0) - m*g*l*np.cos(alpha)

# The robot will lose contact with the ground
condition2 = (m*g*l + 2*m*g*l*np.cos(alpha) + K*alpha**2 + 2*K*alpha*Theta_0) / (m*l**2)

plt.figure()
contourf = plt.contourf(K, Theta_0, omega_star, levels=100, cmap='viridis')
plt.colorbar(label='$\\dot{\\theta}^*$')
contour1 = plt.contour(K, Theta_0, condition1, levels=[0], colors='red')
contour2 = plt.contour(K, Theta_0, condition2 - omega_star**2, levels=[0], colors='blue')

plt.xlabel('$k$')
plt.ylabel('$\\theta_0$')
plt.title('heat map of steady-state walking speed $\\dot{\\theta}^*$')

red_patch = mpatches.Patch(color='red', label='The robot cannot start from rest')
blue_patch = mpatches.Patch(color='blue', label='The robot will lose contact with the ground')
plt.legend(handles=[red_patch, blue_patch])
plt.show()